In [ ]:
%load_ext autoreload
%autoreload 2
from fedflax import train
from models import fetch_vit, NonTrainable, AsymLinear
from data import fetch_data
from utils import load_model, save_model, nnx_norm
import jax, optax, pickle
from flax import nnx
from functools import reduce
from jax import numpy as jnp

n_clients = 4
asymkwargs = {}

## Load Google's ViT as backbone and attach head
Mix flax.linen and flax.nnx

In [ ]:
class Classifier(nnx.Module):
    def __init__(self, key=jax.random.key(0), **asymkwargs):
        super().__init__()
        self.backbone, self.bbparams = fetch_vit()
        self.bbparams = jax.tree.map_with_path(lambda path, p: nnx.Param(p) if not any("LayerNorm" in part.key for part in path) else nnx.BatchStat(p), self.bbparams)
        self.head = AsymLinear(768, 40, key=key, **asymkwargs)

    def __call__(self, x, train=False):
        x = self.backbone.apply({"params": self.bbparams}, x, train=train)
        x = x.reshape(x.shape[0], -1)
        x = self.head(x)
        return x
ell = lambda m, mg, y, x: optax.sigmoid_binary_cross_entropy(m(x, train=True), y).mean() + nnx_norm(nnx.state(m, nnx.Param), n_clients=n_clients)

# Optimizer which applies only to nnx.Param, leaving linen backbone untouched
model = Classifier(**asymkwargs)
lr = optax.warmup_exponential_decay_schedule(1e-4, .1, 4000, 1000, .9, end_value=1e-5)
opt = nnx.Optimizer(
    model,
    optax.chain(
        optax.masked(optax.adam(1e-5), lambda ptree: jax.tree.map_with_path(lambda path, _p: any("bbparams" in str(part) for part in path), ptree)),
        optax.masked(optax.adam(lr), lambda ptree: jax.tree.map_with_path(lambda path, _p: not any("bbparams" in str(part) for part in path), ptree))
    )
)

## Finetune using asymmetries

In [ ]:
# Cityscapes data
ds_train = fetch_data(beta=1., dataset=3, n_clients=n_clients, batch_size=16, skew="label")
ds_val = fetch_data(beta=1., dataset=3, partition="val", n_clients=n_clients, batch_size=16, skew="label")

# Train
err_fn = lambda m, y, x: jnp.mean((m(x, train=False)>0)==(y>0.5))
models, rounds = train(
    model,
    opt,
    ds_train,
    ell, 
    ds_val,
    local_epochs="early",
    n_clients=n_clients,
    max_patience=3,
    rounds="early",
    val_fn=err_fn
)

# Save decoder
save_model(models, "models/celeba_base.pkl")

## Reload & evaluate

In [ ]:
models = load_model(lambda: Classifier(**asymkwargs), "models/celeba_base.pkl")
struct, state, rest = nnx.split(models, (nnx.Param, nnx.BatchStat), ...)
model = nnx.merge(struct, jax.tree.map(lambda p: p.mean(0), state), rest)
ds_val = fetch_data(beta=1., dataset=3, partition="test", n_clients=1, batch_size=16)
vtest_fn = nnx.jit(err_fn)
error = reduce(lambda acc, model: acc + vtest_fn(model, ds_val), model, 0.) / len(models)
print(f"Global test error (aggregated model): {error*100:.2f}%")